<img src="https://cursos.virtual.uniandes.edu.co/isis4219/wp-content/uploads/sites/162/2014/11/cropped-misisheader.png" ><br>
# Machine Learning Techniques - ISIS4219

Intersemestral 2020

### Integrantes:
* Juan José Villegas
* Juan Sebastián Méndez

De acuerdo a los conceptos vistos en clase desarrolle el siguiente ejercicio práctico:

 1. El Internet de las Cosas (IoT) está desempeñando un papel cada vez más importante en nuestras actividades cotidianas, al habilitar la conexión de los objetos que nos rodean con servicios digitales. 
 
 Últimamente, IoT se ha visto afectado por una variedad de botnets, los cuales intentan explotar vulnerabilidades en los protocolos de aplicación que interactúan directamente con los sistemas, provocando violaciones de seguridad. 
 
 Una solución a este tipo de ataques es identificar, de manera automática, eventos maliciosos contra los protocolos de comunicación. En este sentido, ***se desea desarrollar un sistema de detección de intrusos que permita una identificación temprana de estas amenazas***. Sin embargo, el Departamento de Seguridad quiere identificar que ataque específico se está ejecutando para poder implementar los protocolos correspondientes.

Como recomendación general, se espera que para la entrega del taller se aplique la estructura establecida por la metodología CRISP- DM, según corresponda con las caracteristicas de cada conjunto de datos.

# 1. Importando las librerias requeridas para el desarrollo del laboratorio

In [1]:
import pandas as pd
from pandas_profiling import ProfileReport
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.decomposition import TruncatedSVD

# 2. Perfilamiento y entendimiento de los datos

In [2]:
#Importando los datos
diccionario = pd.read_csv('./Punto 1/NUSW-NB15_diccionario.csv', encoding = "ISO-8859-1")
train = pd.read_csv('./Punto 1/UNSW_NB15_training-set.csv', encoding = "ISO-8859-1")
test = pd.read_csv('./Punto 1/UNSW_NB15_testing-set.csv', encoding = "ISO-8859-1")

In [3]:
#Resultado de ProfileReport se encuentra en profile_report.html
#ProfileReport(train)

In [4]:
corr = train.corr()
corr.style.background_gradient(cmap='coolwarm')

<img src="./Imagenes/Correlaciones.png" >

## Notas:
* El label no es necesario porque lo que se quiere es predicir el tipo de ataque. Existe el tipo "normal" que es equivale al label 0. Columna se elimina.
* La columna de Id no es necesaria. Columna se elimina.
* Se observa de la tabla de correlación y del reporte del ProfileReport que existen variables altamente correlacionadas. Se eliminan aquellas columnas que manejan bytes (Ejemplo, dbytes).
* Hay que tranformar los atributos categóricos proto, service y state a números.
* Hay que escalar todas las variable ya que están muy dispersas

# 3. Limpieza y preparación de datos

In [5]:
train.shape

(175341, 45)

In [6]:
# Drop features that are not needed
# We should think of droping highly correlated features 
#features_to_drop = ['label', 'attack_cat', 'ï»¿id', 'sbytes', 'dbytes', 'dloss', 'sloss','dwin', 'tcprtt', 'ct_src_dport_ltm','ct_dst_sport_ltm','is_ftp_login','is_sm_ips_ports']
features_to_drop = ['label', 'attack_cat', 'ï»¿id', 'sbytes', 'dbytes', 'dloss', 'sloss']
X_train, y_train = train.drop(features_to_drop, axis=1), train['attack_cat']
X_test, y_test = test.drop(features_to_drop, axis=1), test['attack_cat']

In [7]:
num_preprocessor = make_pipeline(
    MinMaxScaler(feature_range=(-1,1))
)

categories_preprocessor = make_pipeline(
    OneHotEncoder()
)

preprocessor = make_column_transformer(
    #(num_preprocessor, ('spkts', 'dpkts', 'dinpkt', 'djit', 'trans_depth', 'response_body_len')),
    (num_preprocessor, ('dur','spkts','dpkts','rate','sttl','dttl','sload','dload','sinpkt','dinpkt','sjit', 'djit','swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean', 'trans_depth', 'response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm','ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm', 'is_ftp_login','ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports')),
    (categories_preprocessor, ('proto', 'service', 'state')),
    remainder = 'passthrough'
)

In [8]:
X_train_prepared = preprocessor.fit_transform(X_train)
X_test_prepared = preprocessor.fit_transform(X_test)
X_train_prepared.shape

(175341, 190)

# 4. Modelamiento

In [10]:
#model = SGDClassifier(alpha=0.001, max_iter=10000,n_jobs=-1)

model = SVC(kernel='rbf', C=50, gamma=0.005)

**Sintonización de hiperparámetros**

In [11]:
param_grid = {'C': [1, 5, 10, 50][1, 5, 10, 50],
             'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid,scoring='f1_macro', return_train_score=True,n_jobs=-1)
%time grid.fit(X_train_prepared, y_train)
print(grid.best_params_)

In [ ]:
grid.best_score_

# 5. Predicción

In [ ]:
model = grid.best_estimagrid
yfit = model.predict(X_test_prepared)

# 6. Validación

In [ ]:
print(classification_report(y_test, yfit))